In [8]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import math

In [9]:
conn = sqlite3.connect('DB.db')
c = conn.cursor()

# Problem 1

## Part 1

There is a table 'prices' in a DB with columns 'product' (product names), 'date_start' (dates when price changes) , and 'price' (product prices). A query returns the average price of the newest prices of all products. The result is written into the file.

In [28]:
c.execute("""SELECT AVG(price) FROM
                (SELECT price FROM prices GROUP BY product HAVING MAX(date_start))""")
avg = '%.3f' % c.fetchall()[0][0]
with open ('res1.txt', 'w') as w:
    w.write(avg) 

## Part 2

In [10]:
def price_change(product: str):
    """Returns the list of every date in a ascending order when the product price changed
    with the list of corresponding prices for a chosen product  
    """
    c.execute("SELECT DISTINCT product from prices ORDER BY product")
    names = c.fetchall()
    product_names=[]
    [product_names.append(names[i][0]) for i in range(len(names))]
    if product in product_names:
        c.execute("SELECT date_start, price FROM prices WHERE product=:product ORDER BY date_start", {'product': product})
        alist = c.fetchall()
        return ([alist[i][0] for i in range(len(alist))], [alist[i][1] for i in range(len(alist))])
    else:
        return 'There is no such product'

In [11]:
print(price_change('A'))

(['1970-01-01', '2010-01-21', '2010-08-17', '2010-11-02', '2012-08-27', '2012-12-06', '2013-01-11', '2013-05-02', '2013-05-08', '2013-11-01', '2014-08-30', '2015-12-29', '2016-05-04', '2016-08-14', '2016-09-28', '2017-04-01'], [297.79, 164.68, 558.58, 127.08, 352.13, 0.35, 100.61, 120.5, 260.19, 1747.84, 59.84, 118.27, 550.62, 339.35, 210.19, 111.23])


# Problem 2

## Part 1. Create a table 'revenue'

There is a table 'sales' in a DB with columns 'product' (product names), 'amount' (numbers of product items have been sold) 'date' (sales dates). The following code creates a new table called 'revenue' with the same columns and one additional one also called 'revenue' where revenue is calculated as a product of amount and an actual product price.

In [202]:
prices_dict, sales_dict = {}, {}

for product in product_names:
    c.execute("SELECT date_start, price FROM prices WHERE product=:product", {'product': product})
    prices = c.fetchall()
    product_prices_dict = {}
    for price in prices:
        product_prices_dict[f'{price[0]}'] = f'{price[1]}'
    prices_dict.update({product:product_prices_dict})

for product in product_names:
    c.execute("SELECT date, amount FROM sales WHERE product=:product", {'product': product})
    sales = c.fetchall()
    product_sales_dict = {}
    for sale in sales:
        product_sales_dict[f'{sale[0]}'] = f'{sale[1]}'
    sales_dict.update({product:product_sales_dict})

In [198]:
with conn:
    c.execute("""CREATE TABLE revenue (
                product TEXT,
                date TEXT,
                amount INTEGER,
                revenue REAL
                )""")

In [199]:
%%time
dataset=[]
for product, data in sales_dict.items():
    for date, amount in data.items():
        revenue = float((prices_dict[product][max([dt for dt in prices_dict[product].keys() if dt <= date])]))*int(amount)
        dataset.append((product, date, amount, revenue))
    
query = "INSERT INTO revenue VALUES (?, ?, ?, ?)"
with conn:
     c.executemany(query, dataset)

Wall time: 1.12 s


## Part 2. Calculate standard deviation of product 'B'

In [216]:
c.execute("SELECT revenue FROM revenue WHERE product='B'")
std = np.std([row[0] for row in c.fetchall()])
with open ('res2.txt', 'w') as w:
    w.write('%.2f' % std)

# Problem 3

In [4]:
def lin_reg(product, date_start, date_end):
    """Trains a regression model y=kx+b having dates from a certain range as a predictor
    and product amount as a dependent variable. Returns a regression coefficient (k) and an intercept (b). 
    """
    c.execute("""SELECT date, amount FROM sales
                 WHERE product=:product
                 AND date BETWEEN :date_start AND :date_end
                 ORDER BY date""", {'product': product, 'date_start': date_start, 'date_end': date_end})
    data = c.fetchall()
    X, y = [i for i in range(len(data))], [data[i][1] for i in range(len(data))]

    X_df = pd.DataFrame(data=X, columns=['date'])
    y_df = pd.DataFrame(data=y, columns=['amount'])
    X_train = np.asanyarray(X_df[['date']])
    y_train = np.asanyarray(y_df[['amount']])
    
    LR = LinearRegression().fit(X_train, y_train)
    
    return float('%.4f' % LR.coef_[0][0]), float('%.4f' % LR.intercept_[0])

In [7]:
coef = lin_reg(product='Q', date_start='2019-01-01', date_end='2019-03-31')
with open ('res3.txt', 'w') as w:
    w.write(str(coef[0]) + ' ' + str(coef[1]))

In [236]:
conn.close()